In [1]:
import os
import polars as pl
import dotenv
from tqdm import tqdm
import plotly.graph_objects as go
FOLDER_PATH = os.getenv("FOLDER_PATH")


dotenv.load_dotenv()

Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8
Python-dotenv could not parse statement starting at line 9
Python-dotenv could not parse statement starting at line 10


True

In [2]:
df =  df = pl.read_parquet(f"{FOLDER_PATH}INTC/INTC_2024-07-22.parquet")

In [3]:
df.head()

ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_ct_02,ask_ct_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_ct_03,ask_ct_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_ct_04,ask_ct_04,bid_px_05,ask_px_05,bid_sz_05,ask_sz_05,bid_ct_05,ask_ct_05,bid_px_06,ask_px_06,bid_sz_06,ask_sz_06,bid_ct_06,ask_ct_06,bid_px_07,ask_px_07,bid_sz_07,ask_sz_07,bid_ct_07,ask_ct_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol,date,ts_recv
"datetime[ns, US/Eastern]",u8,u16,u32,str,str,u8,f64,u32,u8,i32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,str,date,"datetime[ns, UTC]"
2024-07-22 04:00:00.019136430 EDT,10,2,8565,"""A""","""N""",0,32.94,300,130,166862,272865,32.94,null,300,0,1,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""INTC""",2024-07-22,2024-07-22 08:00:00.019303292 UTC
2024-07-22 04:00:00.019176414 EDT,10,2,8565,"""A""","""A""",0,33.24,300,130,165991,272870,32.94,33.24,300,300,1,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""INTC""",2024-07-22,2024-07-22 08:00:00.019342405 UTC
2024-07-22 04:00:00.124162129 EDT,10,2,8565,"""A""","""A""",0,33.15,98,130,165583,274848,32.94,33.15,300,98,1,1,null,33.24,0,300,0,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""INTC""",2024-07-22,2024-07-22 08:00:00.124327712 UTC
2024-07-22 04:00:00.124660817 EDT,10,2,8565,"""T""","""N""",0,32.95,60,0,221450,274851,32.94,33.15,300,98,1,1,null,33.24,0,300,0,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""INTC""",2024-07-22,2024-07-22 08:00:00.124882267 UTC
2024-07-22 04:00:00.124660817 EDT,10,2,8565,"""A""","""B""",0,33.03,49,130,221450,274852,33.03,33.15,49,98,1,1,32.94,33.24,300,300,1,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""INTC""",2024-07-22,2024-07-22 08:00:00.124882267 UTC


In [4]:
num_entries_by_publisher = df.group_by("publisher_id").len().sort("len", descending=True)
if len(num_entries_by_publisher) > 1:
    df = df.filter(pl.col("publisher_id") == 41)

In [ ]:
print(num_entries_by_publisher)

In [6]:
df = df.filter(
    (
        (pl.col("ts_event").dt.hour() == 9) & (pl.col("ts_event").dt.minute() >= 30) |
        (pl.col("ts_event").dt.hour() > 9) & (pl.col("ts_event").dt.hour() < 16) |
        (pl.col("ts_event").dt.hour() == 16) & (pl.col("ts_event").dt.minute() == 0)
    )
)

In [7]:
mid_price = (df["ask_px_00"] + df["bid_px_00"]) / 2        
# managing nans or infs, preceding value filling
mid_price = mid_price.fill_nan(mid_price.shift(1))

In [ ]:

# Create figure
fig = go.Figure()

# Add best bid line
fig.add_trace(go.Scatter(
    x=df['ts_event'],
    y=df['bid_px_00'],
    mode='lines',
    name='Best Bid',
    line=dict(color='blue')
))

# Add best ask line  
fig.add_trace(go.Scatter(
    x=df['ts_event'], 
    y=df['ask_px_00'],
    mode='lines',
    name='Best Ask',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=df['ts_event'],
    y=mid_price,
    mode='lines',
    name='Mid Price',
    line=dict(color='black')
))


# Update layout
fig.update_layout(
    title='Order Book and bid/ask',
    xaxis_title='Time',
    yaxis_title='Price',
    showlegend=True
)

fig.show()



In [ ]:
pl.save_parquet(df, f"{FOLDER_PATH}/INTC/INTC_2024-07-22_curated.parquet")

In [ ]:

for stock in tqdm(os.listdir(FOLDER_PATH)[0:1], desc="Stock treatment"):
    for file in tqdm(os.listdir(f"{FOLDER_PATH}/{stock}"), desc="File treatment"):
       
        if pl.col("publisher_id").n_unique() > 1:
            df = df.filter(pl.col("publisher_id") == 41)
        else:
            df = df.filter(pl.col("publisher_id") == 2)
            
            
        if stock == "GOOGL":
            df = df.filter(pl.col("ts_event").dt.hour() >= 13)
            df = df.filter(pl.col("ts_event").dt.hour() <= 20)
        else:
            df = df.filter(
                (
                    (pl.col("ts_event").dt.hour() == 9) & (pl.col("ts_event").dt.minute() >= 30) |
                    (pl.col("ts_event").dt.hour() > 9) & (pl.col("ts_event").dt.hour() < 16) |
                    (pl.col("ts_event").dt.hour() == 16) & (pl.col("ts_event").dt.minute() == 0)
                )
            )
        
        mid_price = (df["ask_px_00"] + df["bid_px_00"]) / 2
        
        # managing nans or infs, preceding value filling
        mid_price = mid_price.fill_nan(mid_price.shift(1)).fill_inf(mid_price.shift(1)).fill_null(mid_price.shift(1))
        df = df.with_columns(mid_price=mid_price)
        df.write_parquet(f"{FOLDER_PATH}/{stock}/{file[:-9]}_curated.parquet")
        






